# Apriori Algorithm Code Implementation

In [1]:
# Importing the libraries and the dataset

import pandas as pd

Market_Data = pd.read_csv('Market_Basket_Dataset.csv',index_col=None, header = None ) # Use your local path here

Market_Data.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,whole wheat pasta,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,soup,light cream,shallot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,frozen vegetables,spaghetti,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Converting the Market Dataset into a nested list
Dataset = []

for index, transaction in Market_Data.iterrows():
    cleaned_transaction = transaction[~transaction.isnull()].tolist()
    Dataset.append(cleaned_transaction)

In [3]:
len(Dataset) # Number of transactions in our dataset

7501

In [5]:
# For the given dataset writing a function to return the list of distinct items in the dataset

def createItem(dataSet):
    
    """
    This function extracts all the unique items from the entire dataset and sorts them (alphabetically)

    Attributes
    ----------
    dataSet : list
        Market dataset
    
    Return Types
    ---------
    frozen Itemlist : list
        frozen list of unique items
    """
    
    itemList = []
    
    for transaction in dataSet:
        for item in transaction:
            if not [item] in itemList:
                
                 # creating unique single lists in Itemlist. ie list of all items
                itemList.append([item])
                
   # itemList.sort()
    
    return list(map(frozenset, itemList))

In [6]:
# Returns an Itemset and dictionary with Support values

def scanData(data, itemsetk, minSupport):
    
    """
    As you know, support is the frequency of a particular item or itemset. 
    In order to calculate the support, we have created a supportDict dictionary.
    This stores the support value for each item set. 
    If the support value is greater than minsupport, we store the itemset in freqItemset. 
    Finally the function returns the ‘freqItemset’ and the ‘supportDict’. 
    
    Attributes
    ----------
    data : list
    List of frozensets

    itemsetk : Frozen set
    Note they can also be 2 item sets such as [a, b] or 3 item sets like []
    
    minSupport : float
    minimum support set by the user
    
    Return Types
    ---------
    freqItemset : Frozenset
    list of itemsets that satisfy the support threshold
    
    supportDict : dict
    Support values of each itemset
    
    """
    
    tempDict = {}
    
    for transaction in data:
        for item in itemsetk:
            if item.issubset(transaction):
                if not item in tempDict: 
                    tempDict[item]=1 # tempDict contains number of all items
                else: 
                    tempDict[item] += 1
    
    numItems = float(len(data))
    freqItemset = []
    supportDict = {}
    
    for key in tempDict:
        support = tempDict[key]/numItems 
        
        if support >= minSupport:
            freqItemset.insert(0,key) # freqItemset contains all frequent items
        supportDict[key] = support # contains support of all items
    
    return freqItemset, supportDict

In [7]:
# Creating Higher order Itemsets

def itemSetGenerator(itemsetk, k):
    
    """
    This function creates itemsets of order k by merging the input itemsets of order k-1)
    
    Attributes
    ----------
    itemsetk: list
        contains a frozen list of itemsets of (k-1)th order   
    k: int
        order of itemset we want to generate   
    
    Return Types
    ---------
    higherOrderitemset : list
        Merged itemsets
        
    """
        
    higherOrderitemset = []
    lenitemsetk = len(itemsetk)
    
    
    for i in range(lenitemsetk):
        for j in range(i+1, lenitemsetk): 
            L1 = sorted(list(itemsetk[i]))[:k-2]
            L2 = sorted(list(itemsetk[j]))[:k-2]
            L1.sort() 
            L2.sort()
            
            # Two frequent itemsets of order k are merged only if their k-1 itemsets are identical
            if L1 == L2:
                higherOrderitemset.append(itemsetk[i] | itemsetk[j]) # Performing set union creates itemset with n+1 items
               
    return higherOrderitemset

In [8]:
def frquentItemsetGeneration(dataSet, minSupport):
    
    """"
    The apriori function generates all the frequent itemsets and the support values for each possible itemset.
    The reason for storing all the support values is that it will be used later for Rule Generation
    
    Attributes
    ----------
    dataSet: list
        The list of all transactions created before creating the functions.
    minSupport: float
        Since minSupport is completed upto the user, we have assumed the minSupport as 0.01 for the Market_Data dataset.
    
    Return Types
    ---------
    freqItemsets : list
         All possible frequent itemsets (for all values of order k)
    supportDict : dict
        support of all the itemsets
        
    """
    itemset1 = createItem(dataSet) # Creating frozenset of items
    
    
    # Generating all the frequent 1-itemsets and the support of those items
    freqItemset1, supportDict = scanData(dataSet, itemset1, minSupport)
    
    freqItemsets = [freqItemset1]
    k = 2 
    
    while (len(freqItemsets[k-2]) > 0): # Incrementing k until we no longer find any kth order itemsets
        
        itemsetk = itemSetGenerator(freqItemsets[k-2], k) # Generating itemsets of order k
        
        # Generating the frequent itemset for the kth order and support for each of these itemsets
        freqItemsetk, supportDictk = scanData(dataSet, itemsetk, minSupport) 
        
        supportDict.update(supportDictk)
        freqItemsets.append(freqItemsetk)
        
        k += 1
    return freqItemsets, supportDict

In [9]:
freqItemsets, supportDict = frquentItemsetGeneration(Dataset, minSupport = 0.01)

In [12]:
def calcConf(freqSet, H, supportDict, bigRuleList, minConf):

    """"
    For each conseq in H, this function generates rules and calculates its confidence
    If the confidence is greater than minconf, we store the rule. We also store their rule consequents in prunedH.
    
    Attributes
    ----------
    freqSet: list
        frequent itemset (We have already generated the frequent itemsets in PART A) 
        eg. frozenset({'french fries', 'mineral water', 'spaghetti'})
        
    H: list
        Combinations of items stored in freqset
        eg. For the freqSet example shown above, H can be 
            [frozenset({'french fries'}), frozenset({'spaghetti'}), frozenset({'mineral water'})]
            or
            [frozenset({'french fries', 'spaghetti'}), 
            frozenset({'french fries', 'mineral water'}), 
            frozenset({'spaghetti', 'mineral water'})]
            
    supportDict: dict
        Support Values of all the possible generated until now
        
    bigRuleList: list
        All our rules will be stored in this list and will be updated every time the function is executed
    
    minConf: float
        minimum confidence

    
    Return Types
    ---------
    prunedH : list
          Contains those consequents whose rules had a confidence higher than minconf
        
    """
        
    prunedH = []
    
    for conseq in H:
        
        conf = supportDict[freqSet]/supportDict[freqSet - conseq] # calculate confidence
        
        if conf >= minConf:
            bigRuleList.append((freqSet-conseq, conseq, conf))
            print(freqSet-conseq, '--->', conseq, 'confidence = ', conf)
            prunedH.append(conseq)
            
    return prunedH

In [13]:
def rulesFromConseq(freqSet, H, supportDict, bigRuleList, minConf):
    
    """"
    This function generates rules for itemsets of order > 2
    
    Attributes
    ----------
    freqSet: list
        frequent itemset
        
    H: list
        Combinations of items stored in freqset
            
    supportDict: dict
        Support Values of all the possible generated until now
        
    bigRuleList: list
        All our rules will be stored in this list and will be updated every time the function is executed
    
    minConf: float
        minimum confidence
        
    """

    m = len(H[0]) # Order of the consequent while generating the rules
         
    H = calcConf(freqSet, H, supportDict, bigRuleList, minConf)
    if len(H)>1: # For len(H)<=1, you cannot generate higher order cadnidates
        
        # creating higher order candidates
        Hmp1 = itemSetGenerator(H, m+1) 
        
        if Hmp1 == []: 
            # Hmp1 will be an empty list if the itemsets in H don't satisfy the condition for merging.
            # Thus higher item set consequent will not be generated and the function will be terminated
            return 0
        
        if (len(Hmp1[0]) < len(freqSet)):
            # Generate rules while the order of the itemsets in Hmp1 is less than the number of items in the frequent itemset
            rulesFromConseq(freqSet, Hmp1, supportDict, bigRuleList, minConf)

In [14]:
def generateRules(freqItemsets, supportDict, minConf):  #supportDict is a dictionary coming from scanData
    
    """"
    For each conseq in H, this function creates a rule and the confidence is calculated. 
    If the confidence is greater than minconf, we store the rule. We also store the rule consequents in prunedH.
    
    Attributes
    ----------
    freqItemsets: list
        Frequent Itemsets generated from PART A
        eg. frozenset({'french fries', 'mineral water', 'spaghetti'})
    supportDict: dict
        Support Dictionary created in PART A
    minConf: float
        minimum confidence set by the user
    
    Return Types
    ---------
    bigRuleList : list
          Contains all the rules whose confidence is greater than minConf
        
    """
    bigRuleList = []
    for i in range(1, len(freqItemsets)): # Only get the sets with two or more items
        for freqSet in freqItemsets[i]:
            H1 = [frozenset([item]) for item in freqSet]  
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportDict, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportDict, bigRuleList, minConf)
    return bigRuleList 

In [15]:
final_rules = generateRules(freqItemsets, supportDict, 0.3)

frozenset({'cooking oil'}) ---> frozenset({'spaghetti'}) confidence =  0.31070496083550914
frozenset({'red wine'}) ---> frozenset({'spaghetti'}) confidence =  0.36492890995260663
frozenset({'fresh bread'}) ---> frozenset({'mineral water'}) confidence =  0.30959752321981426
frozenset({'herb & pepper'}) ---> frozenset({'spaghetti'}) confidence =  0.32884097035040427
frozenset({'grated cheese'}) ---> frozenset({'mineral water'}) confidence =  0.3333333333333333
frozenset({'grated cheese'}) ---> frozenset({'spaghetti'}) confidence =  0.3155216284987277
frozenset({'herb & pepper'}) ---> frozenset({'mineral water'}) confidence =  0.34501347708894875
frozenset({'herb & pepper'}) ---> frozenset({'ground beef'}) confidence =  0.3234501347708895
frozenset({'soup'}) ---> frozenset({'mineral water'}) confidence =  0.45646437994722955
frozenset({'olive oil'}) ---> frozenset({'spaghetti'}) confidence =  0.3481781376518219
frozenset({'cereals'}) ---> frozenset({'mineral water'}) confidence =  0.39896